# Live Air Quality Index (AQI)

Sanjeev - Data-Alchemist

## Load data

In [24]:
#%run ../aqi_dashboard/utils/get_aqi_data.py
import os
import pandas as pd
import utils.queries as q
from utils.get_aqi_data import get_engine, run_query
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv(dotenv_path='.streamlit/')

user = os.getenv("SNOWFLAKE_USER")
password = os.getenv("SNOWFLAKE_PASSWORD")
account = os.getenv("SNOWFLAKE_ACCOUNT")
warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
database = os.getenv("SNOWFLAKE_DATABASE")
schema = os.getenv("SNOWFLAKE_SCHEMA")

engine = create_engine(
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)

In [25]:
df_aqi_per_state = run_query(q.get_aqi_data_per_state)
def get_filtered_monthly_data(df, year, month):
    return df[(df["year"] == year) & (df["month"] == month)].sort_values(by="aqi", ascending=False)

def get_filtered_yearly_data(df, year):
    return df[(df["year"] == year)].sort_values(by="aqi", ascending=False)

In [23]:
import calendar
df_monthly_aqi_per_state = get_filtered_monthly_data(df_aqi_per_state, 2025, 7)
df_yearly_aqi_per_state = get_filtered_yearly_data(df_aqi_per_state, 2025)

df_yearly_aqi_per_state['month_num'] = df_yearly_aqi_per_state['month']
df_yearly_aqi_per_state['month'] = df_yearly_aqi_per_state['month'].apply(lambda x: calendar.month_abbr[int(x)])
df_yearly_aqi_per_state.sort_values(by="month_num", ascending=False)
df_yearly_aqi_per_state

,year,month,state,aqi,pm10_avg,pm25_avg,so2_avg,no2_avg,o3_avg,co_avg,prominent_pollutant,aqi_category,month_num
119,2025,Jul,Jharkhand,361.0,40,26,25,82,5,17,SO2,Very Poor,7
399,2025,May,Chandigarh,348.0,49,31,6,23,28,37,CO,Severe,5
243,2025,Feb,Chandigarh,345.0,48,31,6,23,28,37,CO,Severe,2
354,2025,Mar,Meghalaya,315.0,8,8,6,52,10,3,SO2,Severe,3
256,2025,Feb,Meghalaya,312.0,8,8,6,43,10,3,SO2,Very Poor,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2025,Jun,Sikkim,50.0,14,17,1,2,26,5,CO,Good,6
262,2025,Feb,Sikkim,50.0,14,17,1,2,26,5,CO,Good,2
233,2025,Jan,Sikkim,50.0,14,17,1,2,26,5,CO,Good,1
360,2025,Mar,Sikkim,50.0,14,17,1,2,26,5,CO,Good,3


In [46]:
df_aqi_per_state[df_aqi_per_state['state']=="Uttar_Pradesh"]
df_aqi_per_state[df_aqi_per_state['state_for_map']=="Uttar Pradesh"]

,state,aqi,pm10_avg,pm25_avg,so2_avg,no2_avg,o3_avg,co_avg,prominent_pollutant,aqi_category,state_for_map
12,Uttar_Pradesh,151.0,51,37,22,19,24,29,PM2.5,Very Poor,Uttar Pradesh


Prep for visualizations

In [ ]:
import json
with open("./utils/india_states.geojson", "r") as f:
    india_geo = json.load(f)

## Plots

### Heatmap

In [ ]:
import altair as alt

month_list = df_aqi_data["aqi_month"].unique().tolist()
# selected_month = st.selectbox('Select a year', month_list)
selected_month=7
df_selected_month = df_aqi_data[df_aqi_data.aqi_month == selected_month]
df_selected_month_sorted = df_selected_month.sort_values(by="aqi", ascending=False)

# color_theme_list = ['blues', 'cividis', 'greens', 'inferno', 'magma', 'plasma', 'reds', 'rainbow', 'turbo', 'viridis']
# selected_color_theme = st.selectbox('Select a color theme', color_theme_list)
selected_color_theme="blues"

alt.themes.enable("dark")

def make_heatmap(input_df, input_y, input_x, input_color, input_color_theme):
    heatmap = alt.Chart(input_df).mark_rect().encode(
            y=alt.Y(f'{input_y}:O', axis=alt.Axis(title="Month", titleFontSize=18, titlePadding=15, titleFontWeight=900, labelAngle=0)),
            x=alt.X(f'{input_x}:O', axis=alt.Axis(title="", titleFontSize=18, titlePadding=15, titleFontWeight=900)),
            color=alt.Color(f'max({input_color}):Q',
                             legend=None,
                             scale=alt.Scale(scheme=input_color_theme)),
            stroke=alt.value('black'),
            strokeWidth=alt.value(0.25),
        ).properties(width=900
        ).configure_axis(
        labelFontSize=12,
        titleFontSize=12
        ) 
    # height=300
    return heatmap

heatmap = make_heatmap(df_aqi_data, 'aqi_month', 'state_geojson', 'aqi', selected_color_theme)

alt.Chart(...)

### Choropleth

In [ ]:
import altair as alt
import plotly.express as px
def make_choropleth_india(input_df, input_color_theme, selected_month):
    choropleth = px.choropleth(
        input_df,
        geojson=india_geo,
        locations="state_for_map",
        featureidkey="properties.ST_NM",
        color="aqi",
        color_continuous_scale=input_color_theme,
        range_color=(0, input_df[input_df.month == selected_month]["aqi"].max()),
        
    )
    
    choropleth.update_geos(
        fitbounds="locations", visible=False
    )
    choropleth.update_layout(
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=0, b=0),
        height=700

    )
    #choropleth.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
    return choropleth


alt.Chart(...)